In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import healpy as hp

In [41]:
# Define the HEALPix pixel level and compute the total number of pixels
healpix_level = 4
nside = 2 ** healpix_level
npix = hp.nside2npix(nside)


In [180]:
processed_files = 0

In [181]:
# read processed files
with open('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/processed_2') as f:
    processed_files = f.readlines()
processed_files = [x.split('  ') for x in processed_files]

In [182]:
#processed_files.remove(['\n'])#.remove([''])
# remove everything that is not a number
processed_files_list = [x for i in processed_files for x in i]

In [183]:
processed_files_clean = []
for i in processed_files_list:
    # remove everything that is not a number from the string i
    i = ''.join(filter(str.isdigit, i))
    if i:  # check if i is not an empty string
        #print(int(i))
        processed_files_clean.append(int(i))

In [184]:
# make a list of every healpix pixel level 4 that is not in processed_files_clean
all_files = list(range(npix))
to_process = [x for x in all_files if x not in processed_files_clean]

In [185]:
len(to_process)

2

In [186]:
import os

In [187]:
for left_to_process in to_process:
    finished = os.listdir('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/candidates_50_v2')
    file_name = f'candidates_{left_to_process}.fits'
    if file_name in finished:
        print(f'{file_name} already processed')
        continue

In [188]:
len(to_process)

2

In [189]:
to_process

[1831, 2633]

In [190]:
# save to_process to a csv
to_process = pd.DataFrame(to_process)
to_process.to_csv('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/to_process.csv', index=False)

In [191]:
work_to_do = pd.read_csv('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/to_process.csv')

In [79]:
work_to_do['0'].values

array([ 902, 1793, 1797, 1798, 1799, 1805, 1806, 1807, 1817, 1825, 1828,
       1829, 1830, 1831, 1873, 1886, 1889, 1893, 1894, 1897, 1900, 1903,
       1910, 1913, 1917, 1921, 1924, 1925, 1967, 1972, 1975, 1982, 1985,
       1990, 1997, 1998, 1999, 2006, 2009, 2014, 2015, 2021, 2062, 2065,
       2069, 2070, 2081, 2085, 2086, 2087, 2093, 2094, 2095, 2105, 2113,
       2116, 2153, 2158, 2161, 2167, 2174, 2177, 2181, 2182, 2185, 2188,
       2191, 2198, 2201, 2205, 2239, 2254, 2255, 2257, 2260, 2263, 2270,
       2273, 2278, 2285, 2286, 2287, 2294, 2297, 2333, 2349, 2350, 2351,
       2353, 2357, 2358, 2369, 2373, 2374, 2375, 2381, 2382, 2383, 2428,
       2430, 2441, 2445, 2446, 2449, 2455, 2462, 2465, 2469, 2470, 2473,
       2476, 2479, 2521, 2525, 2527, 2534, 2542, 2543, 2545, 2548, 2551,
       2558, 2561, 2566, 2573, 2574, 2591, 2614, 2615, 2617, 2621, 2633,
       2637, 2638, 2639, 2641, 2645, 2646, 2657, 2661, 2662, 2663, 2689,
       2705, 2709, 2713, 2716, 2717, 2718, 2729, 27

In [171]:
# load first run
first_run = Table.read('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/all_candidates.fits')#pd.read_csv('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/first_run.csv')



In [173]:
# compute 3d velocity by first transforming to galactocentric cartesian and then computing the modulus of the velocity vectior
from astropy.coordinates import SkyCoord, Galactocentric
from astropy import units as u

gc = Galactocentric()
coords = SkyCoord(first_run['ra']*u.deg, first_run['dec']*u.deg, distance=first_run['implied_distance']*u.pc,
                  pm_ra_cosdec=first_run['pmra']*u.mas/u.yr, pm_dec=first_run['pmdec']*u.mas/u.yr, radial_velocity=first_run['VR']*u.km/u.s) 

In [172]:
# # compute 3d velocity by first transforming to galactocentric cartesian and then computing the modulus of the velocity vectior
# from astropy.coordinates import SkyCoord, Galactocentric
# from astropy import units as u

# gc = Galactocentric()
# coords = SkyCoord(first_run['ra'].values*u.deg, first_run['dec'].values*u.deg, distance=first_run['implied_distance'].values*u.pc,
#                   pm_ra_cosdec=first_run['pmra'].values*u.mas/u.yr, pm_dec=first_run['pmdec'].values*u.mas/u.yr, radial_velocity=first_run['VR'].values*u.km/u.s) 

AttributeError: 'Column' object has no attribute 'values'

In [174]:
coords_galactocentric = coords.transform_to(gc)

In [175]:
x, y, z, vx, vy, vz = coords_galactocentric.x, coords_galactocentric.y, coords_galactocentric.z, coords_galactocentric.v_x, coords_galactocentric.v_y, coords_galactocentric.v_z

In [176]:
velocity_total = np.sqrt(vx**2 + vy**2 + vz**2)

In [177]:
first_run['3d_velocity'] = velocity_total
first_run['x'] = x
first_run['y'] = y
first_run['z'] = z
first_run['vx'] = vx
first_run['vy'] = vy
first_run['vz'] = vz


In [137]:
from astropy.table import Table

In [179]:
first_run.write('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/all_candidates_with_velocity.fits', overwrite=True)

In [196]:
candidates = Table.read('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/high_likelihood_candidates2.dat', format='ascii')

In [199]:
candidates = Table.read('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/high_likelihood_candidates.dat', format='ascii')

In [200]:
candidates

col1,col2,col3
int64,float64,float64
3519228788535778176,183.3312353995936,-18.076000678578016
3215231248139929216,79.35198278625761,-1.1496835330325736
4203086469406294144,282.2677426198415,-10.040093666728527
3353379695848965504,104.0138829819836,13.616277696325136
5774843632691792512,246.4167874691072,-81.72772417123953
6097980758553306240,218.66563232931205,-47.25680591159758
6098149666722079360,219.75433455857737,-45.9799651907169
6099772064847615744,220.3096481703587,-43.146178885663645
5230980960610541568,164.7423554873834,-71.60675263557779


: 

In [195]:
# round off ra and dec to 3 decimal places
#candidates['ra'] = np.round(candidates['ra'], 3)
#candidates['dec'] = np.round(candidates['dec'], 3)


# select candidates that can be observed in the southern hemisphere
candidates = candidates[candidates['dec'] < 0]


candidates.write('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/high_likelihood_candidates3.dat', format='ascii.no_header', overwrite=True)